# Lab 1 Monte Carlo Tree Search

In this lab, we'll be using the game connect four, as a vehicle for learning MinMax and Monte Carlo Tree Search.
We'll also introduce concepts, such as state, that'll stay relevant throughout the course.
Expect to lose in connect four to the algorithm at the end of the lab.

## Setup
This section you won't need to edit, but it is worth skimming through—this is where we declare the objects you'll be interacting with througout the lab.

In [1]:
# imports
import random
from typing import List, Tuple
import time
from copy import deepcopy # world -> thought

In [2]:
# world and world model
class State:
    def __init__(self, cols=7, rows=6, win_req=4):
        self.board = [['.'] * cols for _ in range(rows)]
        self.heights = [1] * cols
        self.num_moves = 0
        self.win_req = win_req

    def get_avail_actions(self) -> List[int]:
        return [i for i in range(len(self.board[0])) if self.heights[i] <= len(self.board)]

    def put_action(self, action, agent):
        self.board[len(self.board) - self.heights[action]][action] = agent.name
        self.heights[action] += 1
        self.num_moves += 1

    def is_over(self):
        return self.num_moves >= len(self.board) * len(self.board[0])

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        header  = " ".join([str(i) for i in range(len(self.board[0]))])
        line    = "".join(["-" for _ in range(len(header))])
        board   = [[e for e in row] for row in self.board]
        board   = '\n'.join([' '.join(row) for row in board])
        return  '\n' + header + '\n' + line + '\n' + board + '\n'


In [3]:
# evaluate the utility of a state
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    def diags_pos():
        """Get positive diagonals, going from bottom-left to top-right."""
        for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    def diags_neg():
        """Get negative diagonals, going from top-left to bottom-right."""
        for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
            yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg()) + list(diags_pos())
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


In [4]:
# parrent class for mcts, minmax, human, and any other idea for an agent you have
class Agent:
    def __init__(self, name: str):
        self.name: str = name

    def get_action(self, state: State):
        return random.choice(state.get_avail_actions())

In [5]:
# connecting states and agents
class Game:
    def __init__(self, agents: Tuple[Agent]):
        self.agents = agents
        self.state = State()

    def play(self):
        while utility(self.state) == 0 and not self.state.is_over():
            for agent in self.agents:
                if utility(self.state) == 0 and not self.state.is_over():
                    action = agent.get_action(self.state)
                    self.state.put_action(action, agent)
                    print(self.state)

## Exercise 0: Discuss and Run game
Let's discuss if the `utility` function best belongs to the state or the agent.
Put the state, agent and game class together so that a game is run.

In [6]:
agents = (Agent('O'), Agent('X'))
game = Game(agents)
game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . X . . . .
. . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
. . X . . . .
. . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
. . X . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
O . X . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
O . X . . . .
X . O . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
O . X . . . .
X . O . O . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
O . X . . . .
X X O . O . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
O .

## Exercise 1: Human Agent
Make a child class of `Agent` called `Human`, with the `get_action` method overwritten to take input from you. *hint*: use `int(input())`

In [7]:
class Human(Agent):
  def __init__(self, name):
    super(Human, self).__init__(name)

  def get_action(self, state:State):
    inputa= int(input(" "))
    while (inputa not in state.get_avail_actions()):
      inputa= int(input())
    return inputa


agents = (Agent('O'), Human('X'))
game = Game(agents)
game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .

 0

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X . O . . . .

 0

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . O . . . .
X . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . O . . . .
X . O . . . O

 0

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X . O . . . .
X . O . . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X . O . . . .
X O O . . . O

 0

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
X . O . . . .
X O O . . . O



## Exercise 2: Gekko
Make a child class of `Agent` called `Gekko`, with a `get_action` that is very short sighted (greedy). You can basically do whatever you want here, as long as your output a valid action. You might want to make a `utility` function for the agent, and perhaps some helper functions. Write a two line comment explaining your Gekko's heuristic.

i wanna implement a Gekko that wins if possible. Otherwise he chooses a random action.

In [12]:
class Gekko(Agent):
    def __init__(self, name):
      super(Gekko, self).__init__(name)

    def get_action(self, state:State):
      for a in state.get_avail_actions():
        actiona=self.canWin(a,state);
      if(actiona == None):
          actiona= random.choice(state.get_avail_actions())
      return actiona


    def canWin(self, action, state):
        state1=deepcopy(state)
        state1.put_action(action, self)
        if (utility(state1) ==1):# and self.name=='X'):
          return action
        if (utility(state1) ==-1):# and self.name=='O'):
          return action


agents = (Agent('O'), Gekko('X'))
game = Game(agents)
game.play()




0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . O .
. X . . . O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . O .
. X . . . O X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . O .
. X . O . O X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . O X
. X . O . O X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . O X
. X . O . O X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . O X
. X . O X O X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O

## *Optional exercise: MinMax (useful to have done for exercise 3)*
Make a MinMax agent:

In [14]:
class MinMax(Agent):
  def __init__(self, name):
    super(MinMax, self).__init__(name)
    self.maxDepth=5

  def minmax(self, state, maximazPlayer, depth):
    if isTerminal(state) or depth==0 :
      return utility(state)
    if state.is_over():
      return 0
    if maximazPlayer:
      maxEvaluation=-28938
      for a in state.get_avail_actions():
        newState=deepcopy(state)
        newState.put_action(a, self)
        evaluation=self.minmax(newState,False, depth-1)
        maxEvaluation=max(maxEvaluation, evaluation)
      return maxEvaluation
    else:
      minEvaluation=28938
      for a in state.get_avail_actions():
        newState=deepcopy(state)
        newState.put_action(a, self)
        evaluation=self.minmax(newState, True, depth-1)
        minEvaluation=min(minEvaluation, evaluation)
      return minEvaluation

  def get_action(self, state: State):
     action=random.choice(state.get_avail_actions())
     topeval=0
     for a in state.get_avail_actions():
        newState=deepcopy(state)
        newState.put_action(a, self)
        if self.name=='X':
          max=True
        else:
          max=False
        evaluation=self.minmax(newState, max, self.maxDepth)
        if self.name=='X':
          if(evaluation>topeval):
            topeval=evaluation
            bestAction=a
        else:
          if(evaluation<topeval):
            topeval=evaluation
            bestAction=a
     return bestAction


def isTerminal(state):
  if utility(state)==1 or utility(state)==-1 :
    return True
  else: return False

agents = (MinMax('O'), Agent('X'))
game = Game(agents)
game.play()



0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O X . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O X . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . X
O X . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O . . . . . X
O X . . . . X



## Exercise 3: MCTS
Same but for Monte Carlo Tree Search. See if you can beat it with a `Human`.

In [ ]:
class Node:
     def __init__(self,action, state: State, parent: 'Node' = None ):
         self.children: List['Node'] = []
         self.parent: 'Node' = parent
         self.state: State = state
         #i added this variable to implement UCB1
         self.visits=0
         self.value=0
         self.action=action

In [ ]:
import math
class MCTS(Agent):
      def __init__(self, name):
        self.max_iterations=5
        super(MCTS, self).__init__(name)
        self.maxDepth=5


      def get_action(self, state):
         root = Node(None, state)
         #the MCTS aalgorithm is made of 4 phases:
         for _ in range(self.max_iterations):
            #Selection
            selected= self.selection(root)
            assert selected.state is not None

            #Expansion
            if not isTerminal(selected.state):

                nodeExpanded= self.expand(selected)
                assert nodeExpanded.state is not None
            else:
                nodeExpanded = selected
                assert nodeExpanded.state is not None

            #Simulation
            if self.name=='X':
              max=True
            else:
              max=False
            simulation_result = self.simulation(nodeExpanded.state, max, self.maxDepth)

            #Backpropagation
            self.backpropagation(nodeExpanded, simulation_result)


         bestNode=root.children[0]
         for n in root.children:
          if(n.value> bestNode.value and self.name=="X"):
            bestNode=n
          if(n.value< bestNode.value and self.name=="O"):
            bestNode=n
         best_action = bestNode.action

         return best_action


      def selection(self, node):
        while not isTerminal(node.state) and len(node.children) > 0:
          node = max(node.children, key=ucb1)
        return node

      def expand(self, node):
        if not isTerminal(node.state):
          actions = node.state.get_avail_actions()
          for action in actions:
            next_state=deepcopy(node.state)
            next_state.put_action(action, self)
            new_node = Node(action,next_state, parent=node)
            node.children.append(new_node)
        return random.choice(node.children)

      def simulation(self,state,  maximazPlayer, depth):
        if isTerminal(state) or depth==0 :
          return utility(state)
        if state.is_over():
          return 0
        if maximazPlayer:
          maxEvaluation=-28938
          for a in state.get_avail_actions():
            newState=deepcopy(state)
            newState.put_action(a, self)
            evaluation=self.simulation(newState,False, depth-1)
            maxEvaluation=max(maxEvaluation, evaluation)
          return maxEvaluation
        else:
          minEvaluation=28938
          for a in state.get_avail_actions():
            newState=deepcopy(state)
            newState.put_action(a, self)
            evaluation=self.simulation(newState, True, depth-1)
            minEvaluation=min(minEvaluation, evaluation)
          return minEvaluation
        return 0;

      def backpropagation(self, node, value):
         while node is not None:
           node.visits += 1
           node.value += value
           node = node.parent


def isTerminal(state):
  if utility(state)==1 or utility(state)==-1 or state.is_over() :
      return True
  else: return False

def ucb1(node):
    if node.visits == 0:
        return float("inf")
    exploitation = node.value / node.visits
    exploration = math.sqrt(2 * math.log(node.parent.visits) / node.visits)
    return exploitation + exploration



agents = (MCTS('O'), Agent('X'))
game = Game(agents)
game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O X . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O X . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O X . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O X . . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O X X . . . X


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
O . . . . . .
O X X . . . X



## *Optional exercise: Dynamic Programming*
Then use dynamic programming to make your AI more efficient. You can use the class below (or not)

In [15]:
class TranspositionTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.vals = [None] * size

    def board_str(self, state: State):
        return ''.join([''.join(c) for c in state.board])

    def put(self, state: State, utility: float):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        self.vals[idx] = (bstr, utility)

    def get(self, state: State):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        stored = self.vals[idx]
        if stored is None:
            return None
        if stored[0] == bstr:
            return stored[1]
        else:
            return None

In [16]:
#does not work
def MCTSDP(Agent):

    def __init__(self, name):
        self.max_iterations=5
        super(MCTS, self).__init__(name)
        self.maxDepth=5
        self.transposition_table = TranspositionTable()


    def get_action(self, state):
         root = Node(None, state)
         #the MCTS aalgorithm is made of 4 phases:
         for _ in range(self.max_iterations):
            #Selection
            selected= self.selection(root)
            assert selected.state is not None

            #Expansion
            if not isTerminal(selected.state):

                nodeExpanded= self.expand(selected)
                assert nodeExpanded.state is not None
            else:
                nodeExpanded = selected
                assert nodeExpanded.state is not None

            #Simulation
            if self.name=='X':
              max=True
            else:
              max=False
            simulation_result = self.simulation(nodeExpanded.state, max, self.maxDepth)

            #Backpropagation
            self.backpropagation(nodeExpanded, simulation_result)


         bestNode=root.children[0]
         for n in root.children:
          if(n.value> bestNode.value and self.name=="X"):
            bestNode=n
          if(n.value< bestNode.value and self.name=='O'):
            bestNode=n
         best_action = bestNode.action

         return best_action



    def simulate(state, transposition_table, maximazPlayer, depth):
        cached_utility = self.transposition_table.get(state)
        if cached_utility is not None:
            return cached_utility

        if isTerminal(state) or depth==0 :
              self.transposition_table.put(state, utility(state))
              return utility(state)
        if state.is_over():
              self.transposition_table.put(state, 0)
              return 0
        if maximazPlayer:
              maxEvaluation=-28938
              for a in state.get_avail_actions():
                newState=deepcopy(state)
                newState.put_action(a, self)
                evaluation=self.simulation(newState,False, depth-1)
                maxEvaluation=max(maxEvaluation, evaluation)
              self.transposition_table.put(state, maxEvaluation)
              return maxEvaluation
        else:
              minEvaluation=28938
              for a in state.get_avail_actions():
                newState=deepcopy(state)
                newState.put_action(a, self)
                evaluation=self.simulation(newState, True, depth-1)
                minEvaluation=min(minEvaluation, evaluation)
              self.transposition_table.put(state, minEvaluation)
              return minEvaluation
        return 0


    def selection(self, node):
        while not isTerminal(node.state) and len(node.children) > 0:
          node = max(node.children, key=ucb1)
        return node

    def backpropagation(self, node, value):
         while node is not None:
           node.visits += 1
           node.value += value
           node = node.parent

    def expand(self, node):
        if not isTerminal(node.state):
          actions = node.state.get_avail_actions()
          for action in actions:
            next_state=deepcopy(node.state)
            next_state.put_action(action, self)
            new_node = Node(action,next_state, parent=node)
            node.children.append(new_node)
        return random.choice(node.children)

def isTerminal(state):
  if utility(state)==1 or utility(state)==-1 or state.is_over() :
      return True
  else: return False

def ucb1(node):
    if node.visits == 0:
        return float("inf")
    exploitation = node.value / node.visits
    exploration = math.sqrt(2 * math.log(node.parent.visits) / node.visits)
    return exploitation + exploration

In [17]:

agents = (MCTSDP('O'), Agent('X'))
game = Game(agents)
game.play()

AttributeError: ignored